In [1]:
import ast
import configparser
import pandas as pd
import psycopg2

In [2]:
path = 'outputs/by-product'

In [3]:
config = configparser.ConfigParser()

In [4]:
config.read('config.ini')

['config.ini']

In [5]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = config['ro-prod']['host'],
            database = config['ro-prod']['name'],
            user = config['ro-prod']['user'],
            password = config['ro-prod']['pass'])
    
    conn.autocommit = True
    cur = conn.cursor()
    
    return conn, cur

In [6]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [7]:
def get_product_name(barcode, cur):
    barcode_iqvia = barcode.zfill(15)
    
    query = f" \
        SELECT IP.\"PRODUTO\" \
        FROM iqvia_produtos IP \
        WHERE IP.\"EAN\" = '{barcode_iqvia}';"            
    cur.execute(query)
    res = cur.fetchone()

    if res:
        return ''.join(res).strip()
    else:
        query = f" \
            SELECT DSP.\"name\" \
            FROM data_storeproduct DSP \
            WHERE DSP.barcode = '{barcode}' \
            ORDER BY DSP.last_change DESC;"            
        cur.execute(query)
        res = cur.fetchone()

        if res:
            return ''.join(res).strip()
        else:
            return barcode

In [8]:
def translate(product_names, barcodes):
    barcodes = ast.literal_eval(barcodes)
    products = [product_names[barcode] for barcode in barcodes]

    return ', '.join(products)

---

In [9]:
df = pd.read_csv(f"{path}/association_rules.csv")

In [10]:
df = df[df['lift'] >= 1].sort_values(by = 'lift', ascending = False)

In [11]:
df.shape

(21817, 5)

In [12]:
print(f"Total barcodes: {df['antecedent'].apply(lambda x: len(ast.literal_eval(x))).sum()}")

Total barcodes: 93174


In [13]:
barcodes = []

for item in df['antecedent'].values.tolist():
    barcodes.extend(ast.literal_eval(item))

barcodes = list(set(barcodes))

In [14]:
print(f"Unique barcodes: {len(barcodes)}")

Unique barcodes: 252


In [15]:
conn, curr = open_connection()

In [16]:
%%time
product_names = {}

for barcode in barcodes:
    product_names[barcode] = get_product_name(barcode, curr)

CPU times: user 52.1 ms, sys: 7.88 ms, total: 60 ms
Wall time: 1min 17s


In [17]:
close_connection(conn, curr)

In [18]:
df['antecedent'] = df['antecedent'].apply(lambda x: translate(product_names, x))

In [19]:
df['consequent'] = df['consequent'].apply(lambda x: translate(product_names, x))

In [20]:
df.to_csv(f"{path}/association_rules_translated.csv", index = False)

In [21]:
barcodes_not_found = [item for item in product_names.values() if item.isnumeric()]

In [22]:
print(f"Unknown barcodes: {len(barcodes_not_found)}")

Unknown barcodes: 27


In [23]:
df = pd.DataFrame(data = barcodes_not_found, columns = ['barcode']).reset_index(drop = True)

In [24]:
df.to_csv(f"{path}/barcodes_not_found.csv", index = False)